In [1]:
import pandas as pd
import requests
from config import z_key

## MTA station info

In [2]:
# read information from MTA
mta_data = pd.read_csv('http://web.mta.info/developers/data/nyct/subway/Stations.csv')
mta_data.head()

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude
0,1,1,R01,BMT,Astoria,Astoria - Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034
1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843
2,3,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479
3,4,4,R05,BMT,Astoria,Broadway,Q,N W,Elevated,40.761820,-73.925508
4,5,5,R06,BMT,Astoria,36 Av,Q,N W,Elevated,40.756804,-73.929575


## Use package to get zipcode and address
https://pypi.org/project/geopy/

In [3]:
from uszipcode import Zipcode
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True) 
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="search zip code and address")

zipcode = []
address = []
for index, row in mta_data.iterrows():
    # Search zipcode within 2 miles, ordered from closest to farthest
    tmp = search.by_coordinates(row['GTFS Latitude'], row['GTFS Longitude'], radius=2, returns=1)
    if tmp:
        zipcode.append(tmp[0].zipcode)
        
    # Search address based on lan&lon
    location = geolocator.reverse(f"{row['GTFS Latitude']}, {row['GTFS Longitude']}")
    if location.address:
        address.append(location.address)
        
mta_data['Zip Code'] = zipcode
mta_data['Address'] = address
                                  
pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)

# To fetch more house info, create more combination of zip code and address option
from operator import methodcaller
new_add_list = list(map(methodcaller("split", ", "), mta_data["Address"]))
mta_data["Zip Code2"] = [elem[-2] for elem in new_add_list]
mta_data["Address2"] = [f"{elem[0]}, {elem[1]}" for elem in new_add_list]
mta_data["Address3"] = [elem[0] for elem in new_add_list]
mta_data["Address4"] = [elem[1] for elem in new_add_list]
                                  
# final mta data with address infomation
mta_data.to_csv("Resources/mta_data_with_zip_address.csv")

# Use zillow api to fetch house info

Over 1,000 calls in a day, API calls will be blocked

run up to 20 API calls on one page at one time

## GetDeepSearch-Results API
* http://www.zillow.com/webservice/GetDeepSearchResults.htm
    

In [5]:
from xmljson import badgerfish as bf
from xml.etree.ElementTree import fromstring
import json
from json import dumps

zillow_url = "http://www.zillow.com/webservice/GetDeepSearchResults.htm"
params = { 'zws-id': z_key }
results_fetch_key = "{http://www.zillow.com/static/xsd/SearchResults.xsd}searchresults"

option = {"comb1": ["Stop Name", "Zip Code"],
          "comb2": ["Address2", "Zip Code"],
          "comb3": ["Address3", "Zip Code"],
          "comb4": ["Address4", "Zip Code"],
          "comb5": ["Stop Name", "Zip Code2"],
          "comb6": ["Address2", "Zip Code2"],
          "comb7": ["Address3", "Zip Code2"],
          "comb8": ["Address4", "Zip Code2"]}

zpid = []
latitude = []
longitude = []
bathrooms = []
bedrooms = []
zipcode = []
price = []
lotSizeSqFt = []
usecode = []
for key, value in option.items():
    
    for index, row in mta_data.iterrows():
        params['address'] = row[value[0]]
        params['citystatezip'] = row[value[1]]

        response = requests.get(zillow_url, params=params)
        json_data = bf.data(fromstring(response.content))    
        
        if json_data[results_fetch_key]["message"]["code"]["$"] == 0:
            results = json_data[results_fetch_key]['response']['results']['result']
            if not isinstance(results, list):
                results = [results]

            for house in results:
                zpid.append(house['zpid']['$'])
                latitude.append(house['address']['latitude']['$'])
                longitude.append(house['address']['longitude']['$'])
                usecode.append(house['useCode']['$'])

                if 'zipcode' in house['address'] and '$' in house['address']['zipcode']:
                    zipcode.append(house['address']['zipcode']['$'])
                else:
                    zipcode.append(0)

                if 'bathrooms' in house:
                    bathrooms.append(house['bathrooms']['$'])
                else:
                    bathrooms.append(0)

                if 'bedrooms' in house:
                    bedrooms.append(house['bedrooms']['$'])
                else:
                    bedrooms.append(0)

                if 'amount' in house['zestimate'] and '$' in house['zestimate']['amount']:
                    price.append(house['zestimate']['amount']['$'])
                else:
                    price.append(0)

                if 'lotSizeSqFt' in house:
                    lotSizeSqFt.append(house['lotSizeSqFt']['$'])
                else:
                    lotSizeSqFt.append(0)           

df = pd.DataFrame(columns=['zpid'])
df['zpid'] = zpid
df['latitude'] = latitude
df['longitude'] = longitude
df['usecode'] = usecode
df['bathrooms'] = bathrooms
df['bedrooms'] = bedrooms
df['zipcode'] = zipcode
df['price'] = price
df['lotSizeSqFt'] = lotSizeSqFt
    
df.to_csv(f"Resources/house_data.csv")


# Data process
* remove duplicated
* remove home without price
* Calculate neareat Station

In [13]:
from scipy.spatial import distance
unique_df = df.drop_duplicates(subset='zpid')
unique_df = unique_df[unique_df['price'] != 0]

mat = distance.cdist(
        unique_df[['latitude', 'longitude']],
        mta_data[['GTFS Latitude', 'GTFS Longitude']],
        metric = 'euclidean')
distance_df = pd.DataFrame(mat, index=unique_df['zpid'], columns=mta_data['Station ID'])

arr = distance_df.values
unique_df['Nearest Station Index']  = [list(i).index(i.min()) for i in arr]
unique_df['Nearest Station Lat'] = [mta_data.loc[i, 'GTFS Latitude'] for i in unique_df['Nearest Station Index']]
unique_df['Nearest Station Lon'] = [mta_data.loc[i, 'GTFS Longitude'] for i in unique_df['Nearest Station Index']]
unique_df['Borough'] = [mta_data.loc[i, 'Borough'] for i in unique_df['Nearest Station Index']]

from math import sin, cos, sqrt, atan2, radians

def convert_latlon(row):
    R = 6373.0
    
    stoplat = radians(row['Nearest Station Lat'])
    stoplon = radians(row['Nearest Station Lon'])
    
    homelat = radians(row['latitude'])
    homelon = radians(row['longitude'])
    
    dlon = homelon - stoplon
    dlat = homelat - stoplat
    
    a = sin(dlat / 2)**2 + cos(stoplat) * cos(homelat) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance_km = R * c
    distance_m = distance_km * 0.621371
    
    return distance_m

unique_df['distance_miles'] = unique_df.apply(convert_latlon, axis=1)

unique_df.to_csv("Resources/final_house_data.csv")


In [14]:
unique_df.shape


(11465, 14)

In [15]:
unique_df.head()


,zpid,latitude,longitude,usecode,bathrooms,bedrooms,zipcode,price,lotSizeSqFt,Nearest Station Index,Nearest Station Lat,Nearest Station Lon,Borough,distance_miles
0,31947881,40.780026,-73.916672,SingleFamily,1.0,3,11105,1288952,2500,0,40.775036,-73.912034,Q,0.421744
1,94721614,40.770830,-73.903180,MultiFamily2To4,1.0,1,11105,550596,0,0,40.775036,-73.912034,Q,0.547056
2,94721483,40.770830,-73.903180,MultiFamily2To4,1.0,2,11105,679430,0,0,40.775036,-73.912034,Q,0.547056
3,31947885,40.779837,-73.916406,SingleFamily,1.0,3,11105,1398531,2500,0,40.775036,-73.912034,Q,0.403068
4,31943243,40.768852,-73.900892,Triplex,0.0,0,11105,1293072,2200,0,40.775036,-73.912034,Q,0.723042
